In [1]:
########## 실제 프로젝트 #########
import requests
from bs4 import BeautifulSoup

# user_table.csv 파일 가져오기 
import pandas as pd
cols = ['user_name', 'url']
sample_members = pd.read_csv("D:/온이/스터디/데이터분석스터디/webscraping_basic/user_table.csv")

In [2]:
sample_members

,user_id,user_name,user_url
0,0,jeremie-choo,/profile/jeremie-choo
1,1,atxbrbn,/profile/atxbrbn
2,2,sethdow,/profile/sethdow
3,3,duy-viet-nguyen,/profile/duy-viet-nguyen
4,4,thewhiskeyninja,/profile/thewhiskeyninja
...,...,...,...
4744,4744,joseg,/profile/joseg
4745,4745,shortraspy,/profile/shortraspy
4746,4746,teamcrutch,/profile/teamcrutch
4747,4747,valar,/profile/valar


In [4]:
import requests
from bs4 import BeautifulSoup
import re

ratings_df = []
headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/98.0.4758.102 Safari/537.36"}

for i in range(len(sample_members)):
    url = sample_members["user_url"][i]
    # 사용지의 Taste 페이지 url
    url = "https://distiller.com" + url + "/tastes"
    res = requests.get(url, headers=headers)
    res.raise_for_status()
    soup = BeautifulSoup(res.text, "lxml")
    
    # 사용자의 전체 리뷰 수 구하기
    rating_num = soup.select_one("#main-content > div > div.center-column > div > div.desktop-profile-nav > div > div.user-statistics > div.statistic.tastes-link > a > span").get_text()
    rating_num = int(rating_num.split()[0])

    # 사용자 리뷰의 전체 페이지 수 구하기
    total_page = (rating_num // 10) + 1

    for j in range(1, total_page + 1):
        url = url + "?page={}".format(j)
        res = requests.get(url, headers=headers)
        res.raise_for_status()
        soup = BeautifulSoup(res.text, "lxml")
        
        # "div" 태그의 클래스가 "taste user-taste js-carousel"로 시작하는 아이템 가져오기
        items = soup.find_all("li", attrs={"class":re.compile("^taste user-taste js-carousel")})

        for item in items:
            # 해당 술의 url 구하기
            alcohol_url = item.find("a")["href"]
            # 술 이름 구하기
            alcohol_name = item.find("h3", attrs={"class":"mini-headline name truncate-line"}).get_text()
            # 평점이 있으면 float 형태로 구하고, 없으면 0으로 저장
            if item.find("div", attrs={"class":"rating-display__value"}) != None:
                rating = item.find("div", attrs={"class":"rating-display__value"}).get_text()
                rating = float(rating)
            else:
                rating = 0.0
            # 해당 리뷰의 좋아요 수 구하기. 없으면 0으로 저장
            likes_cnt = item.find("button", attrs={"class":"like-count js-show-modal"}).get_text().split()
            if likes_cnt != []:
                likes_cnt = int(likes_cnt[0])
            else:
                likes_cnt = 0
            # 텍스트 리뷰 구하기. 없으면 empty string.
            if item.find("div", attrs={"class":"body"}) != None:
                text_review = item.find("div", attrs={"class":"body"}).get_text()
            else:
                text_review = ""
            # 술 종류 구하기
            alcohol_type = item["class"][-1].split("-")[0]
            
            row = [sample_members['user_id'][i], alcohol_name, alcohol_type, rating, likes_cnt, alcohol_url, text_review]
            ratings_df.append(row)

In [5]:
# DataFrame으로 변환 후 csv파일로 저장
ratings_df = pd.DataFrame(ratings_df, columns=["user_id", "alcohol_name", "alcohol_type", "rating", "likes_cnt", "alcohol_url", "text_review"])
ratings_df.to_csv('ratings_tb.csv', index=False, encoding='utf-8')


In [6]:
ratings_df

,user_id,alcohol_name,alcohol_type,rating,likes_cnt,alcohol_url,text_review
0,0,Johnnie Walker Green Label 15 Year,whiskey,5.00,0,/spirits/johnnie-walker-green-label-15-year,
1,1,Port Charlotte 10 Year First Edition,whiskey,4.00,0,/spirits/port-charlotte-the-ten-year-old,
2,1,Buffalo Trace Bourbon,whiskey,3.00,0,/spirits/buffalo-trace-bourbon,
3,1,Booker's Bourbon,whiskey,3.00,0,/spirits/booker-s-bourbon,
4,1,Angel's Envy Bourbon Finished in Port Wine Bar...,whiskey,3.00,0,/spirits/angel-s-envy-bourbon,
...,...,...,...,...,...,...,...
120028,4745,Compass Box Orchard House,whiskey,4.00,0,/spirits/compass-box-orchard-house,
120029,4745,Glenmorangie Quinta Ruban Port Cask Finish 14 ...,whiskey,4.75,0,/spirits/glenmorangie-quinta-ruban-port-cask-f...,
120030,4745,Peerless Rye Whiskey Single Barrel - Humidor,whiskey,4.00,0,/spirits/peerless-rye-whiskey-single-barrel-hu...,
120031,4745,Smooth Ambler Contradiction Rye,whiskey,5.00,0,/spirits/smooth-ambler-contradiction-rye,
